In [3]:
#some of the preliminary commands for assembly before analysis
#seqtk sample -s 828 oil_R1.fastq 5000000 > pesub1.fastq
#seqtk sample -s 828 oil_R2.fastq 5000000 > pesub2.fastq
#seqtk sample -s 828 oilMP_S4_L001_R1_001.fastq 1500000 > mpsub1.fastq
#seqtk sample -s 828 oilMP_S4_L001_R2_001.fastq 1500000 > mpsub2.fastq
#cat mpsub1.fastq
#mkdir seqtk_out
#mv mpsub* seqtk_out/
#mv pesub* seqtk_out/
#cd seqtk_out/
#multiqc .
#fastqc mpsub1.fastq 
#fastqc mpsub2.fastq && fastqc pesub1.fastq && fastqc pesub2.fastq 
#platanus_trim -i pesub1.fastq && platanus_trim -i pesub2.fastq 
#platanus_trim -i pesub1.fastq 
#echo "pesub1.fastq" > pe.fofn
#echo "pesub2.fastq" > pe.fofn
#platanus_trim -i pe.fofn
#platanus_internal_trim -i mp.fofn 
#fastqc mpsub1.fastq.int_trimmed && fastqc mpsub2.fastq.int_trimmed && fastqc pesub1.fastq.trimmed && fastqc pesub2.fastq.trimmed 
multiqc .
#platanus assemble -o platanus -f *trimmed
#platanus scaffold -o platanus -c platanus_contig.fa -b platanus_contigBubble.fa -IP1 *.trimmed -OP2 .int_trimmed
#platanus scaffold -o platanus -c platanus_contig.fa -b platanus_contigBubble.fa -IP1 pesub1.fastq.trimmed pesub2.fastq.trimmed -OP2 mpsub1.fastq.int_trimmed mpsub2.fastq.int_trimmed
#platanus gap_close -o platanus -c platanus_scaffold.fa -IP1 ../trimmed_reads/pesub1.fastq.trimmed ../trimmed_reads/pesub2.fastq.trimmed -OP2 ../trimmed_reads/mpsub1.fastq.int_trimmed ../trimmed_reads/mpsub2.fastq.int_trimmed

!pip install biopython


Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 2.4 MB/s eta 0:00:00m eta 0:00:010:01:01


In [ ]:
from Bio import SeqIO
def analyze_contigs(file_path):
    # read contig file
    contigs = list(SeqIO.parse(file_path, "fasta"))
    
    # count total contig number
    total_contigs = len(contigs)
    
    # sum length of all contigs
    total_length = sum(len(record.seq) for record in contigs)
    
    # find length of longest contig
    max_length = max(len(record.seq) for record in contigs)
    
    # calculate N50
    sorted_lengths = sorted([len(record.seq) for record in contigs], reverse=True)
    cumulative_sum = 0
    for length in sorted_lengths:
        cumulative_sum += length
        if cumulative_sum >= total_length / 2:
            n50 = length
            break
    
    return {
        "total number of contigs": total_contigs,
        "total length of contigs": total_length,
        "length of longest contig": max_length,
        "N50": n50
    }

contigs_file = "platanus_contig.fasta"

# contig analysis
results = analyze_contigs(contigs_file)
print(results)

In [ ]:
def count_gaps(sequence):
    gaps = []
    current_gap_start = None
    
    for i, char in enumerate(sequence):
        if char == 'N':
            if current_gap_start is None:
                current_gap_start = i
        elif current_gap_start is not None:
            gaps.append((current_gap_start, i - 1))
            current_gap_start = None
    
    if current_gap_start is not None:
        gaps.append((current_gap_start, len(sequence) - 1))
    
    return gaps
    
def analyze_scaffolds(file_scaffolds):
    # read files with FASTA-scaffolds and scaffolds with closed gaps 
    scaffolds = list(SeqIO.parse(file_scaffolds, "fasta"))
    #closed_gaps = list(SeqIO.parse(file_closed_gaps, "fasta"))
    
    # count total scaffold number
    total_scaffolds = len(scaffolds)
    
    # sum length of all scaffolds
    total_length = sum(len(record.seq) for record in scaffolds)
    
    # length of longest scaffold
    max_length = max(len(record.seq) for record in scaffolds)
    
    # N50
    sorted_lengths = sorted([len(record.seq) for record in scaffolds], reverse=True)
    cumulative_sum = 0
    for length in sorted_lengths:
        cumulative_sum += length
        if cumulative_sum >= total_length / 2:
            n50 = length
            break
            
    longest_scaffold = next(record for record in scaffolds if len(record.seq) == max_length)
    num_gaps = len(count_gaps(longest_scaffold.seq))
    gap_lengths = sum([gap[1] - gap[0] + 1 for gap in count_gaps(longest_scaffold.seq)])
    
    return {
        "total number of scaffolds": total_scaffolds,
        "total scaffolds length": total_length,
        "length of longest scaffold": max_length,
        "N50": n50,
        "number of gaps in longest scaffold": num_gaps,
        "total gap length in longest scaffold": gap_lengths
    }

file_scaffolds = "platanus_scaffold.fasta"
#file_closed_gaps = "platanus_gapClosed.fa"

results = analyze_scaffolds(file_scaffolds)
print(results)


In [ ]:
def analyze_scaffolds(file_scaffolds):
    #scaffolds = list(SeqIO.parse(file_scaffolds, "fasta"))
    closed_gaps = list(SeqIO.parse(file_closed_gaps, "fasta"))
    
    total_scaffolds = len(closed_gaps)
    
    total_length = sum(len(record.seq) for record in closed_gaps)
    
    max_length = max(len(record.seq) for record in closed_gaps)
    
    sorted_lengths = sorted([len(record.seq) for record in closed_gaps], reverse=True)
    cumulative_sum = 0
    for length in sorted_lengths:
        cumulative_sum += length
        if cumulative_sum >= total_length / 2:
            n50 = length
            break
    
    longest_scaffold = next(record for record in closed_gaps if len(record.seq) == max_length)
    with open('longest.fasta', 'w') as f:
        SeqIO.write(longest_scaffold, f, 'fasta')
    num_gaps = len(count_gaps(longest_scaffold.seq))
    gap_lengths = sum([gap[1] - gap[0] + 1 for gap in count_gaps(longest_scaffold.seq)])
    
    return {
        "total scaffold number": total_scaffolds,
        "total scaffold length": total_length,
        "length of longest scaffold": max_length,
        "N50": n50,
        "number of gaps in longest scaffold": num_gaps,
        "total gap length in longest scaffold": gap_lengths,
        "longest scaffold": longest_scaffold
    }

#file_scaffolds = "platanus_scaffold.fa"
file_closed_gaps = "platanus_gapClosed.fasta"

results = analyze_scaffolds(file_closed_gaps)
print(results)